In [10]:
import pandas, numpy as np
import xml, itertools, re

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
from html.parser import HTMLParser
pars =HTMLParser()
#pars.unescape

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,27,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
unique =  casos.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
diagnosis.loc[unique, 'dfg'].to_excel('Venezolanas/DiagnosisFiltered.xlsx')

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [208]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

In [4]:
import xml.etree.ElementTree as ET


In [5]:
def getEpicrisis(casoID, registrosByCase):
    return registros[(registros.Caso == casoID) & 
                     (registros.Asunto.str.contains('Epicrisis', case = False) == True)]

In [109]:
def getDataFromCase(case):
    """
    Get some information from the case
    """
    data = {}
    r = ET.fromstring(case.RegistroXML)
    try:
        data['FC']=r.find('.//row[@NombreCampo="FrecuenciaCardiaca"]').get('ValorCampo')
        data['FR']=r.find('.//row[@NombreCampo="FrecuenciaRespiratoria"]').get('ValorCampo')
        data['Presion']=r.find('.//row[@NombreCampo="Presion"]').get('ValorCampo')
        data['Temperatura']=r.find('.//row[@NombreCampo="Temperatura"]').get('ValorCampo')
        data['Talla']=r.find('.//row[@NombreCampo="Talla"]').get('ValorCampo')
        data['Peso']=r.find('.//row[@NombreCampo="Peso"]').get('ValorCampo')
        return data
    except:
        return {}

In [50]:
floatParse = '[0-9]*\.?[0-9]+'
allEqual = lambda s: all([i == j for i, j in  itertools.combinations(s, r = 2)])

def parseGPCA_and_fum(s):
    parsedGPCA = re.findall('G\s*([0-9]+)\s*P\s*([0-9]+)\s*C\s*([0-9]+)\s*A\s*([0-9]+)', registroString)
    parsedFUM = re.findall('FUM: [0-9]+/[0-9]+/[0-9]+', registroString, flags = re.IGNORECASE)
    return {'fum' : len(parsedFUM) > 0,
            'GPCA' : len(parsedGPCA) > 0, 
            'fum_OK' : allEqual(parsedFUM),
            'GPCA_OK' : allEqual(parsedGPCA),
            'fum_Data' : parsedFUM[0] if parsedFUM else [],
            'parsedGPCA' : parsedGPCA[0] if parsedGPCA else []}

In [119]:
import tqdm
casos_wo_epi = 0
casos_several_epi = 0
results ={}
for i,c in tqdm.tqdm_notebook(enumerate(registros.groupby('Caso')), total = len(casos)):
    if i % 100 == 0 and (i < 1000 or i % 1000 == 0):
        print(i)
    
    epicrisis= getEpicrisis(c[0], c[1])
    if len(epicrisis) == 0:    
        casos_wo_epi += 1
    elif len(epicrisis) >= 2:    
        casos_several_epi += 1
    else:
        stringRegistro = epicrisis.RegistroXML.iloc[0]
        results[c[0]] = parseGPCA_and_fum(stringRegistro)
        data = {}
        for cc in c[1].iterrows():
            data = getDataFromCase(cc[1])
            if data:
                break
        results[c[0]].update(data)
    #I am not looking at echo information
    #Search also for "ultimo parto"
    #Search for height, weight and vital signs (can they change?) in the rest of the report. They are a row field in HTML.

0
100
200
300
400
500
600
700
800
900
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000



In [120]:
print('Missing epi: %d' % casos_wo_epi)
print('Several epi: %d' % casos_several_epi)

Missing epi: 9693
Several epi: 437


In [121]:
pd = pandas.DataFrame.from_dict(results, orient = 'index')
np.sum(pd.fum_OK), np.sum(pd.fum), np.sum(pd.GPCA_OK), np.sum(pd.GPCA), len(pd) 

(3940, 3940, 3940, 3940, 3940)

In [125]:
pd.Presion.count()

3824

In [102]:
for c in r.findall('row'):
    print(c, c.items())

<Element 'row' at 0x12b3bba10> dict_items([('NombreCampo', 'TipoConsulta'), ('ValorCampo', 'OBSTETRICA'), ('NombreTabla', 'CamposTexto')])
<Element 'row' at 0x12b3bb890> dict_items([('NombreCampo', 'PlanAdministradora'), ('ValorCampo', 'DEPARTAMENTO ADMINISTRATIVO DISTRITAL DE SALUD (DADIS) / DADIS CARTAGENA SISBENIZADO'), ('NombreTabla', 'CamposTexto')])
<Element 'row' at 0x12b3bbe50> dict_items([('NombreCampo', 'MotivoConsulta'), ('ValorCampo', 'TENGO FIEBRE Y DOLOR DE CABEZA&quot;'), ('NombreTabla', 'CamposTexto')])
<Element 'row' at 0x12b3bb210> dict_items([('NombreCampo', 'EnfermedadActual'), ('ValorCampo', ' PACIENTE DE 16 A&#209;OS DE EDAD CON CUADRO CLINICO DE 15 DIAS DE EVOLUCION CONSISTENTE EN FIEBRE NO CUANTIFICADA ASOCIADO A DEBILIDAD, DISURIA, POLAQUIURIA, TENESMO, ORINA  COLOR  AMARILLA.PACIENTE REFIERE QUE EL DIA DE AYER SE CAYO DE SUS PROPIA ALTURA.POR LO CUAL CONSULTA.'), ('NombreTabla', 'CamposTexto')])
<Element 'row' at 0x12b3bb190> dict_items([('NombreCampo', 'Frecu

In [100]:
help(c)

Help on Element in module xml.etree.ElementTree object:

class Element(builtins.object)
 |  Element(tag, attrib={}, **extra)
 |  
 |  An XML element.
 |  
 |  This class is the reference implementation of the Element interface.
 |  
 |  An element's length is its number of subelements.  That means if you
 |  want to check if an element is truly empty, you should check BOTH
 |  its length AND its text attribute.
 |  
 |  The element tag, attribute names, and attribute values can be either
 |  bytes or strings.
 |  
 |  *tag* is the element name.  *attrib* is an optional dictionary containing
 |  element attributes. *extra* are additional element attributes given as
 |  keyword arguments.
 |  
 |  Example form:
 |      <tag attrib>text<child/>...</tag>tail
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |  
 |  __delitem__(self, index)
 |  
 |  __getitem__(self, index)
 |  
 |  __init__(self, tag, attrib={}, **extra)
 |  
 |  __len__(self)
 |  
 |  __repr__(self)
 |  
 |  __setit

In [87]:
r = registros.loc[393965]

r = xml.dom.minidom.parseString(stringRegistro) #r.RegistroXML)
print(r.toprettyxml())

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD89414" CentroA="01" CodigoDiagnosticoRelacionado1="Z321" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="N771" IdPaciente="106843" Modulo="03" Padre="AD89414" PlanAdm="118" Prestador="73083700" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  1-&amp;amp;gt;LITIASISRENAL
7-&amp;amp;gt;G3P2C0A0
MENARQUIA12A&amp;#209;OS
INICIODEVIDASEXUAL14A&amp;#209;OS
INICIODEVIDAOBSTETRICA15A&amp;#209;OS
COMPA&amp;#209;EROSSEXUALES3
FUM25/2/1514SEMANASHOY
FECHADEULTIMOPARTO9/12/2011

ECOGRAFIA17/4/157SEMNOSEOBSERVAEMBRION
ECOGRAFIA12/5/1510SEMEXTRAPOLADAHOY13.1SEM

NOPARACLINICOSDECONTROLPRENATAL
- FAMILIARES:
  DIABETES - HIPERTENSA MADRE
- AL&amp;#201;RGICOS:
  
- QUIR&amp;#218;RGICOS:"/>
	<row NombreCampo="AplicaCuidadoEnfermeria" NombreTabl

In [88]:
for c in r.childNodes:
    print(c)
    for cc in c.childNodes:
        print(cc, cc.attributes['NombreCampo'].value)

<DOM Element: C at 0x12b45ca50>
<DOM Element: row at 0x12b45c9b0> Antecedentes
<DOM Element: row at 0x12b45c910> AntecedentesHTML
<DOM Element: row at 0x12b45cd70> AplicaCuidadoEnfermeria
<DOM Element: row at 0x12b45c690> AsuntoHistoriaPadre
<DOM Element: row at 0x12b45ce10> Cama
<DOM Element: row at 0x12b45caf0> CambiosDePosicion
<DOM Element: row at 0x12b45ceb0> CodigoDiagnosticoPrincipal
<DOM Element: row at 0x12b45cf50> CodigoDiagnosticoPrincipalSalida
<DOM Element: row at 0x12b45c870> CodigoDiagnosticoRealcionado3
<DOM Element: row at 0x12b17e050> CodigoDiagnosticoRelacionado1
<DOM Element: row at 0x12b17e0f0> CodigoDiagnosticoRelacionado1Salida
<DOM Element: row at 0x12b17e190> CodigoDiagnosticoRelacionado2
<DOM Element: row at 0x12b17e230> CodigoDiagnosticoRelacionado2Salida
<DOM Element: row at 0x12b17e2d0> CodigoDiagnosticoRelacionado3Salida
<DOM Element: row at 0x12b17e370> CodigoHistoriaPadre
<DOM Element: row at 0x12b17e410> CodigoPlanAdministradora
<DOM Element: row at 0x1

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string

In [223]:
rString = registros.RegistroXML[465351]
register = ET.fromstring(rString)

In [236]:
r = xml.dom.minidom.parseString(rString)
print(r.toprettyxml())

<?xml version="1.0" ?>
<C Asunto="Evoluci&amp;#243;n" Caso="AD129559" CentroA="01" CodigoDiagnosticoRelacionado1="Z370" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O839" IdPaciente="118111" Modulo="01" Padre="464998" PlanAdm="100" Prestador="9068204" Raiz="94" Registro="106">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="&amp;lt;P&amp;gt;PACIENTE DE 20 A&amp;#209;OS CON IDX DE:&amp;lt;BR&amp;gt;1. PARTO VAGINAL ASISTIDO (18/11/15) 8:22 PM&amp;lt;BR&amp;gt;2. RNUV&amp;lt;BR&amp;gt;3. PUERPERIO INMEDIATO&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;FC: 78 LPM FR: 17 RPM TA: 110/70 T: 36&amp;lt;BR&amp;gt;&amp;lt;/P&amp;gt;
&amp;lt;P&amp;gt;PACIENTE CONSCIENTE, ALERTA Y ORIENTADA, ENREGULAR ESTADO GENERAL, PALIDEZ GENERALIZADA, PUPILAS ISOC&amp;#211;RICAS NORMOREACTIVAS A LA LUZ, MUCOSAS SECAS Y P&amp;#193;LIDAS, CUELLO SIM&amp;#201;TRICO M&amp;#211;VIL, SIN ADENOPATIAS NI INGURGITACI&amp;#211;N YUGULAR. T&amp;#211;RAX SIMETRICO, EXPANSIB

In [120]:
i = 57
print(i, v[i].attributes['NombreCampo'].value,v[i].nodeValue)
print (" ".join(t.nodeValue for t in v[i].childNodes if t.nodeType == t.TEXT_NODE))

57 Analisis None



In [136]:
from html.parser import HTMLParser
pars =HTMLParser()
pars.unescape(s)

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


'PACIENTE DE 27 AÑOS CON EMBARAZO DE 8 SEMANAS POR FUM + AMENAZA DE ABORTO VS  ABORTO INCOMPLETO, ACTUALMENTE HEMODINAMICAMENTE ESTABLE, AFEBRIL, CON SIGNOS VITALES DENTRO DE PARAMETROS NORMALES, SIN SIGNOS DE SIRS NI IRRITACION PERITONEAL, TOLERANDO OXIGENO AMBIENTE Y VIA ORAL. SE DECIDE INGRESAR A LA PACIENTE PARA REALIZACION DE PARACLINICOS QUE CONFIRMEN DIAGNOSTICO Y REVALORAR.'